In [13]:
import pandas as pd
tracks_df = pd.read_parquet(r"C:\Users\tkeit\OneDrive\Documents\GitHub\bt4222grp9\datasets\parquet datasets\tracks\tracks_Keith.parquet")
tracks_df

,track_uri,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name,inside_playlists
201788,spotify:track:3Kka62A05ETUXW6kl0xJtm,All Fairy Tail Openings by PelleK,spotify:album:70ULSBLEHj1lBuLJhBkKSc,PelleK,spotify:artist:1qLwXsUtyW2Ba2Iotg4gE3,93071,25,Yumeiro Graffiti,"[597838, 325315, 898419]"
201789,spotify:track:49kEbr0L0P1ibeVZ2S6Ej3,All Fairy Tail Openings by PelleK,spotify:album:70ULSBLEHj1lBuLJhBkKSc,PelleK,spotify:artist:1qLwXsUtyW2Ba2Iotg4gE3,91000,26,The Starting Sky,"[597838, 898419]"
201790,spotify:track:2jWvXNrhfPzSpOHndtLcrK,All Fairy Tail Openings by PelleK,spotify:album:70ULSBLEHj1lBuLJhBkKSc,PelleK,spotify:artist:1qLwXsUtyW2Ba2Iotg4gE3,91812,28,Ashita Wo Narase,"[597838, 325315, 898419]"
201791,spotify:track:1AHr7bJlQMy7I3JhcJmdCE,All Fairy Tail Openings by PelleK,spotify:album:70ULSBLEHj1lBuLJhBkKSc,PelleK,spotify:artist:1qLwXsUtyW2Ba2Iotg4gE3,92652,29,Never-end Tale,"[597838, 325315, 898419]"
201792,spotify:track:4tJd85C4DVl6w0DtAUhmSn,Crossing Field,spotify:album:6snkpeTZFimJsbuQBHdxiO,NateWantsToBattle,spotify:artist:0Vb15td3iKkAzdGD5Sj9Ky,92178,32,Crossing Field,"[597838, 481546]"
...,...,...,...,...,...,...,...,...,...
252231,spotify:track:3rcVJVqhuuKbVppaLXrxD5,Linger,spotify:album:44rvrZI6NG38wyyoXQMsVx,Freedom Fry,spotify:artist:195hFqaTDENqLCcG8uGtM7,201136,64,Linger,[553669]
252232,spotify:track:5GVYw2giD7mJWiEoje0BvU,Loveless,spotify:album:6faROkNNmOizrqvaaiq30y,La Bouquet,spotify:artist:2uDY1ixxYwWPw7LXQiROrs,249171,70,Loveless,[553669]
252233,spotify:track:4YQ3wmxU4UJMRBWvGPMJIG,Perfect Enemy,spotify:album:1Rqqezljtnm2UuaObgmP5X,Tilian,spotify:artist:2hj2E6c3lMHm5QlqsQFI04,207137,71,True,[553669]
252234,spotify:track:0J72AmGZYBnGjScXT38JcO,Long Walk Home,spotify:album:2yWzqYipS7h2bDzA9ojotI,Scenic Route to Alaska,spotify:artist:4tUP4gz64sNmLHP1M2eLI5,154920,139,"Again, Again",[553669]


In [6]:
import os
import pandas as pd
import pickle #for saving/loading progress
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Load environment variables
load_dotenv()
SPOTIPY_CLIENT_ID = os.getenv("SPOTIPY_CLIENT_ID")
SPOTIPY_CLIENT_SECRET = os.getenv("SPOTIPY_CLIENT_SECRET")

# Authenticate with Spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="c68416b36aeb4eaabca398101322e5b4",
    client_secret= "44702a064cf740f19ee00de860a7d67a"))

# Helper: Batch iterator
def batch(iterable, n=1):
    """Yield successive n-sized batches from list."""
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

# Load your DataFrame of tracks (if you have it saved before)
# tracks_df = pd.read_csv("your_tracks_file.csv")  # Uncomment if needed

# Get list of track URIs and album URIs from your DataFrame
track_uris = tracks_df["track_uri"].tolist()
album_uris = tracks_df["album_uri"].tolist()

##############################################
# PART 1: Process Tracks with Resume Support #
##############################################

# Create batches for track URIs (Spotify API supports up to 50 per request)
track_batches = list(batch(track_uris, n=50))
total_track_batches = len(track_batches)

# Check if there's an existing progress file for tracks
if os.path.exists("track_details_progress.pkl"):
    with open("track_details_progress.pkl", "rb") as f:
        all_track_details = pickle.load(f)
    # Assuming each batch returns 50 tracks (except possibly the last),
    # we can estimate how many batches are done.
    processed_track_batches = len(all_track_details) // 50
    print(f"Resuming track processing from batch {processed_track_batches + 1} out of {total_track_batches}...")
else:
    all_track_details = []
    processed_track_batches = 0
    print(f"Starting track processing from batch 1 out of {total_track_batches}...")

# Process remaining track batches
for i, track_batch in enumerate(track_batches[processed_track_batches:], start=processed_track_batches + 1):
    print(f"Processing track batch {i}/{total_track_batches}...")
    response = sp.tracks(track_batch)
    all_track_details.extend(response["tracks"])
    # Save progress every 10 batches (adjust as needed)
    if i % 10 == 0:
        with open("track_details_progress.pkl", "wb") as f:
            pickle.dump(all_track_details, f)
        print("Saved track details progress.")

# Save one more time after completing all batches
with open("track_details_progress.pkl", "wb") as f:
    pickle.dump(all_track_details, f)
print("Track processing completed.\n")

##############################################
# PART 2: Process Albums with Resume Support #
##############################################

# For album details, we only need unique album URIs.
unique_album_uris = list(set(album_uris))
album_batches = list(batch(unique_album_uris, n=20))
total_album_batches = len(album_batches)

# Check if there's an existing progress file for albums
if os.path.exists("album_details_progress.pkl"):
    with open("album_details_progress.pkl", "rb") as f:
        album_details_dict = pickle.load(f)
    # Estimate number of processed batches.
    # (This assumes each complete batch returns 20 albums; the last batch might be smaller.)
    processed_album_batches = len(album_details_dict) // 20
    print(f"Resuming album processing from batch {processed_album_batches + 1} out of {total_album_batches}...")
else:
    album_details_dict = {}
    processed_album_batches = 0
    print(f"Starting album processing from batch 1 out of {total_album_batches}...")

# Process remaining album batches
for i, album_batch in enumerate(album_batches[processed_album_batches:], start=processed_album_batches + 1):
    print(f"Processing album batch {i}/{total_album_batches}...")
    response = sp.albums(album_batch)
    for album in response["albums"]:
        # Skip if album is None or its URI is missing
        if album is None or album.get("uri") is None:
            continue
        album_details_dict[album["uri"]] = album
    # Save progress every 5 batches (adjust as needed)
    if i % 5 == 0:
        with open("album_details_progress.pkl", "wb") as f:
            pickle.dump(album_details_dict, f)
        print("Saved album details progress.")

# Save one more time after completing all album batches
with open("album_details_progress.pkl", "wb") as f:
    pickle.dump(album_details_dict, f)
print("Album processing completed.\n")


Starting track processing from batch 1 out of 1009...
Processing track batch 1/1009...
Processing track batch 2/1009...
Processing track batch 3/1009...
Processing track batch 4/1009...
Processing track batch 5/1009...
Processing track batch 6/1009...
Processing track batch 7/1009...
Processing track batch 8/1009...
Processing track batch 9/1009...
Processing track batch 10/1009...
Saved track details progress.
Processing track batch 11/1009...
Processing track batch 12/1009...
Processing track batch 13/1009...
Processing track batch 14/1009...
Processing track batch 15/1009...
Processing track batch 16/1009...
Processing track batch 17/1009...
Processing track batch 18/1009...
Processing track batch 19/1009...
Processing track batch 20/1009...
Saved track details progress.
Processing track batch 21/1009...
Processing track batch 22/1009...
Processing track batch 23/1009...
Processing track batch 24/1009...
Processing track batch 25/1009...
Processing track batch 26/1009...
Processing 

In [12]:
###########################################
# PART 3: Combine Track and Album Metadata #
###########################################

track_metadata = []
for track in all_track_details:
    # Skip if track or its album URI is None
    if track is None or track["album"].get("uri") is None:
        continue
    album_uri = track["album"]["uri"]
    album_info = album_details_dict.get(album_uri, {})
    track_metadata.append({
        "track_uri": track["uri"],
        "track_name": track["name"],
        "track_popularity" : track["popularity"],
        "album_name": album_info.get("name", None),
        "album_release_date": album_info.get("release_date", None),
        #"track_release_date" : album_info.get("release_date_precision", None),
    })

# Save the final metadata to CSV
metadata_df = pd.DataFrame(track_metadata)
metadata_df.to_csv("track_metadata.csv", index=False)

print("Metadata processing completed and saved to 'track_metadata.csv'.")

NameError: name 'all_track_details' is not defined

In [12]:
# Convert the list of dictionaries into a DataFrame
track_metadata_df = pd.DataFrame(track_metadata)
track_metadata_df.head()

,track_uri,track_name,track_popularity,album_name,album_release_date
0,spotify:track:3Kka62A05ETUXW6kl0xJtm,Yumeiro Graffiti,0,All Fairy Tail Openings by PelleK,2016-09-13
1,spotify:track:49kEbr0L0P1ibeVZ2S6Ej3,The Starting Sky,0,All Fairy Tail Openings by PelleK,2016-09-13
2,spotify:track:2jWvXNrhfPzSpOHndtLcrK,Ashita Wo Narase,0,All Fairy Tail Openings by PelleK,2016-09-13
3,spotify:track:1AHr7bJlQMy7I3JhcJmdCE,Never-end Tale,0,All Fairy Tail Openings by PelleK,2016-09-13
4,spotify:track:4tJd85C4DVl6w0DtAUhmSn,Crossing Field,28,Crossing Field,2016-07-14


In [8]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# # 1. Standardize track popularity
scaler = StandardScaler()
track_metadata_df["track_popularity_scaled"] = scaler.fit_transform(track_metadata_df[["track_popularity"]])

def release_date_to_era(release_date):
    """
    Categorizes a song's release date into a decade-based era.

    Arguments:
    - reads in the release_date of the song (of type string) -> format: YYYY-MM-DD, YYYY-MM, or YYYY

    Ideology:
    - cateogrizes the release date into the following eras:
        - 50s: 1950-1959
        - 60s: 1960-1969
        - 70s: 1970-1979
        - 80s: 1980-1989
        - 90s: 1990-1999
        - 2000s: 2000-2009
        - 2010s: 2010-2019
        - 2020s: 2020-2029
        - Future Releases: after 2029
        - None if invalid or missing
    
    Returns:
    - the era the song was released in (of type string)
    """
    try:
        # Convert release_date to standardized year format
        year = pd.to_datetime(release_date, errors='coerce').year

        if pd.isnull(year):
            return None # to handle missing values

        # Categorize into eras
        if year < 1950:
            return "Pre-1950s"
        elif 1950 <= year < 1960:
            return "1950s Rock & Roll"
        elif 1960 <= year < 1970:
            return "1960s Classic Rock & Motown"
        elif 1970 <= year < 1980:
            return "1970s Disco & Hard Rock"
        elif 1980 <= year < 1990:
            return "1980s Pop & New Wave"
        elif 1990 <= year < 2000:
            return "1990s Alternative & Hip-Hop Boom"
        elif 2000 <= year < 2010:
            return "2000s Digital Pop & Emo"
        elif 2010 <= year < 2020:
            return "2010s Streaming & Trap"
        elif 2020 <= year < 2030:
            return "2020s Present Era"
        else:
            return "Future Music"
    
    except Exception as e:
        return None # to catch any unexpected errors



NameError: name 'track_metadata_df' is not defined

In [18]:
# Ensure album_release_date column exists and is properly formatted
metadata_df["album_era"] = metadata_df["album_release_date"].apply(release_date_to_era)

# Display the first few rows
metadata_df[["album_release_date", "album_era"]].head()

display(metadata_df)


,track_uri,track_name,track_popularity,album_name,album_release_date,album_era
0,spotify:track:3Kka62A05ETUXW6kl0xJtm,Yumeiro Graffiti,0,All Fairy Tail Openings by PelleK,2016-09-13,2010s Streaming & Trap
1,spotify:track:49kEbr0L0P1ibeVZ2S6Ej3,The Starting Sky,0,All Fairy Tail Openings by PelleK,2016-09-13,2010s Streaming & Trap
2,spotify:track:2jWvXNrhfPzSpOHndtLcrK,Ashita Wo Narase,0,All Fairy Tail Openings by PelleK,2016-09-13,2010s Streaming & Trap
3,spotify:track:1AHr7bJlQMy7I3JhcJmdCE,Never-end Tale,0,All Fairy Tail Openings by PelleK,2016-09-13,2010s Streaming & Trap
4,spotify:track:4tJd85C4DVl6w0DtAUhmSn,Crossing Field,28,Crossing Field,2016-07-14,2010s Streaming & Trap
...,...,...,...,...,...,...
50441,spotify:track:3rcVJVqhuuKbVppaLXrxD5,Linger,54,Linger,2016-11-29,2010s Streaming & Trap
50442,spotify:track:5GVYw2giD7mJWiEoje0BvU,Loveless,12,Loveless,2016-11-04,2010s Streaming & Trap
50443,spotify:track:4YQ3wmxU4UJMRBWvGPMJIG,True,0,Perfect Enemy,2015-11-27,2010s Streaming & Trap
50444,spotify:track:0J72AmGZYBnGjScXT38JcO,"Again, Again",19,Long Walk Home,2016-09-23,2010s Streaming & Trap


In [ ]:
# from sklearn.preprocessing import StandardScaler
# import pandas as pd

# # Assume track_metadata_df already has "track_popularity" and "album_release_date"

# # 1. Standardize track popularity
# scaler = StandardScaler()
# track_metadata_df["track_popularity_scaled"] = scaler.fit_transform(track_metadata_df[["track_popularity"]])

# # 2. Define a function to convert release dates to eras (e.g., 70s, 80s, etc.)
# def release_date_to_era(release_date):
#     try:
#         # Convert release_date to datetime; this handles formats like "YYYY", "YYYY-MM-DD", etc.
#         year = pd.to_datetime(release_date, errors='coerce').year
#         if pd.isnull(year):
#             return None
#         if 1970 <= year < 1980:
#             return "70s"
#         elif 1980 <= year < 1990:
#             return "80s"
#         elif 1990 <= year < 2000:
#             return "90s"
#         elif 2000 <= year < 2010:
#             return "00s"
#         elif 2010 <= year < 2020:
#             return "10s"
#         else:
#             return "Other"
#     except Exception as e:
#         return None

# # Apply the function to create a new column for era
# track_metadata_df["album_era"] = track_metadata_df["album_release_date"].apply(release_date_to_era)
# track_metadata_df.head()